There are two techniques implemented in order to find emission and/or absorption lines in a Spectrum1D spectrum.

The first technique is find_lines_threshold that will find lines by thresholding the flux based on a factor applied to the spectrum uncertainty. The second technique is find_lines_derivative that will find the lines based on calculating the derivative and then thresholding based on it. Both techniques return an QTable that contains columns line_center, line\_type and line\_center\_index.

In [1]:
import numpy as np
from astropy.modeling import models
import astropy.units as u
from specutils import Spectrum1D, SpectralRegion
from specutils.manipulation import extract_region
from specutils.fitting import find_lines_threshold
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from astropy.wcs import WCS
import warnings
with warnings.catch_warnings():  # Ignore warnings
    warnings.simplefilter('ignore')

In [2]:
# OUR PN
hdu = fits.open("../Spectra-lamostdr7/spec-56581-VB031N50V1_sp08-218.fits")
hdudata = hdu[0].data
wl = hdudata[2]
Flux = hdudata[0]
inve_var = hdudata[1]
sigma = 1 / np.sqrt(inve_var)

In [3]:
import warnings
from specutils.manipulation import noise_region_uncertainty
from astropy.nddata import StdDevUncertainty

In [4]:
ha_region = SpectralRegion((6563-50)*u.AA, (6563+50)*u.AA)
ha_region

Spectral Region, 1 sub-regions:
  (6513.0 Angstrom, 6613.0 Angstrom) 

In [5]:
# Defining units astropy
rel_flux = u.def_unit('Relative~flux')
rel_flux.decompose()

Unit("Relative~flux")

In [6]:
lamb = wl * u.AA 
flux = Flux * rel_flux
Sigma = StdDevUncertainty(sigma * rel_flux)
spec = Spectrum1D(spectral_axis=lamb, flux=flux, uncertainty=Sigma) 

**Adding uncertanting**

In [7]:
#spec = noise_region_uncertainty(spec, noise_region = sigma)

In [8]:
ha_spec = extract_region(spec, ha_region)

$\bf{Finding~the~line}$

In [12]:
with warnings.catch_warnings():  # Ignore warnings
    warnings.simplefilter('ignore')
    halpha_lines = find_lines_threshold(ha_spec, noise_factor = 3)

In [13]:
halpha_lines

line_center,line_type,line_center_index
Angstrom,,
float32,str8,int64
6561.45458984375,emission,32


# #SNR

In [14]:
from specutils.analysis import snr
snr(spec) 

<Quantity 3.4997768>